In [ ]:
import json

In [ ]:
with open("./word2type.json","r",encoding='utf-8') as f:
            word2type = json.load(f)
len(word2type)

In [ ]:
with open("./db2vgg.json","r",encoding='utf-8') as f:
        db2vgg = json.load(f)

In [ ]:
with open("./yolo2db.json","r",encoding='utf-8') as f:
        yolo2db = json.load(f)

In [ ]:
with open("./words.json","r",encoding='utf-8') as f:
        words= json.load(f)

In [ ]:
with open("./word2files.json","r",encoding='utf-8') as f:
        word2files= json.load(f)
len(word2files)

In [ ]:
with open("./word2clip.json","r",encoding='utf-8') as f:
        word2clip= json.load(f)
len(word2clip)

In [ ]:
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial
import time
import json

import subprocess
from subprocess import PIPE, Popen
import os, shutil  
from tqdm import tqdm
import re
import requests
import numpy as np

def worker(word, i):
    print("第{}个词:{}".format(i,word))
    
    try:
        our_word = word.replace(' ', '_')
        results = word2clip[word]
        path = word2files[word][0]
        path = path.split(word)[0]
        
        params = {}
        params['path'] = path.replace('./','../../') + word

        # YOLO
        yolo_flag = True
        
        try:
            r = requests.get(url='http://127.0.0.1:5001/test', params=params) # YOLO server
            yolo_results = json.loads(r.text)

            for index,yolo_result in enumerate(yolo_results):
                for one_yolo_result in yolo_result:
                    if word2type[our_word] in yolo2db[one_yolo_result]:
                        results[0][index] += 3.0 # Filter images based on score difference
                        yolo_flag = False
                        break
                        
        except Exception as e1:
            print('{} skip yolo'.format(word))

        # VGG
        params['path'] = path.replace('./','../') + word
        
        if yolo_flag:
            if db2vgg[word2type[our_word]] != []:    
                r1 = requests.get(url='http://127.0.0.1:5000/test', params=params) # VGG server
                vgg_results = json.loads(r1.text)
                for index,vgg_result in enumerate(vgg_results):
                    for one_vgg_result in vgg_result:
                        if one_vgg_result in db2vgg[word2type[our_word]]:
                            results[0][index] += 1.5
                            break
        return our_word, results[0], i
    
    except Exception as e:
        results = word2clip[word]
        print('{} error:'.format(word), e)
        return our_word, results[0], i

def collectMyResult(result):
    word2final[result[0]] = result[1]
    if(int(result[2]) %5000 == 0):
        print("Got result {}".format(result))
        
    
def abortable_worker(func, *args, **kwargs):
    timeout = 60
    p = ThreadPool(1)
    res = p.apply_async(func, args=args)
    try:
        out = res.get(timeout)  # Wait timeout seconds for func to complete.
        return out
    except multiprocessing.TimeoutError:
        print("Aborting due to timeout")
        raise

if __name__ == "__main__":
    tic = time.time()
    pool = multiprocessing.Pool(processes=3,maxtasksperchild=1)
    
    word2final = {} # final score
    with open("./word2final2_188.json","r",encoding='utf-8') as f:
        word2final = json.load(f)

    featureClass = [[words[i], i] for i in tqdm(range(0,len(words),1))] #list of arguments
    for f in featureClass:
        abortable_func = partial(abortable_worker, worker, timeout=60)
        pool.apply_async(abortable_func, args=f,callback=collectMyResult)
    pool.close()
    pool.join()
    
    print(len(word2final))
    
    
    toc = time.time()
    shijian = toc-tic
    print("time consuming:",shijian)

    with open("./word2final.json","w",encoding='utf-8') as f:
            json.dump(word2final,f)